### Problem Descripition 

In 2012, URL shortening service Bitly partnered with the US government website USA.gov to provide a feed of anonymous data gathered from users who shorten links ending with .gov or .mil.

The text file comes in JSON format and here are some keys and their description. They are only the most important ones for this task.

|key| description |
|---|-----------|
| a|Denotes information about the web browser and operating system|
| tz | time zone |
| r | URL the user come from |
| u | URL where the user headed to |
| t | Timestamp when the user start using the website in UNIX format |
| hc | Timestamp when user exit the website in UNIX format |
| cy | City from which the request intiated |
| ll | Longitude and Latitude |

In the cell, I tried to provide some helper code for better understanding and clearer vision

-**HINT**- Those lines of code may be not helping at all with your task.

In [1]:
# I will try to retrieve one instance of the file in a list of dictionaries
import json
records = [json.loads(line) for line in open('C:/Users/Lenovo/Downloads/Documents/usa.gov_click_data.json')]
# Print the first occurance
print(records[0])
print("rec_num:", len(records))

{'a': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.78 Safari/535.11', 'c': 'US', 'nk': 1, 'tz': 'America/New_York', 'gr': 'MA', 'g': 'A6qOVH', 'h': 'wfLQtf', 'l': 'orofrog', 'al': 'en-US,en;q=0.8', 'hh': '1.usa.gov', 'r': 'http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/wfLQtf', 'u': 'http://www.ncbi.nlm.nih.gov/pubmed/22415991', 't': 1333307030, 'hc': 1333307037, 'cy': 'Danvers', 'll': [42.576698, -70.954903]}
rec_num: 12


## Required

Write a script can transform the JSON files to a DataFrame and commit each file to a sparete CSV file in the target directory and consider the following:

        

In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.DataFrame.from_dict(records,orient='columns')
df.head()

,a,c,nk,tz,gr,g,h,l,al,hh,r,u,t,hc,cy,ll
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,1,America/New_York,MA,A6qOVH,wfLQtf,orofrog,"en-US,en;q=0.8",1.usa.gov,http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/...,http://www.ncbi.nlm.nih.gov/pubmed/22415991,1333307030,1333307037,Danvers,"[42.576698, -70.954903]"
1,GoogleMaps/RochesterNY,US,0,America/Denver,UT,mwszkS,mwszkS,bitly,NaN,j.mp,http://www.AwareMap.com/,http://www.monroecounty.gov/etc/911/rss.php,1331923249,1308262393,Provo,"[40.218102, -111.613297]"
2,Mozilla/4.0 (Windows NT 6.1; MSIE 8.0; Windows...,US,1,America/New_York,DC,xxr3Qb,xxr3Qb,bitly,en-US,1.usa.gov,http://t.co/03elZC4Q,http://boxer.senate.gov/en/press/releases/0316...,1333407030,1333407035,Washington,"[38.9007, -77.043098]"
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,BR,0,America/Sao_Paulo,27,zCaLwp,zUtuOu,alelex88,pt-br,1.usa.gov,direct,http://apod.nasa.gov/apod/ap120312.html,1333507030,1333507044,Braz,"[-23.549999, -46.616699]"
4,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,0,America/New_York,MA,9b6kNl,9b6kNl,bitly,"en-US,en;q=0.8",bit.ly,http://www.shrewsbury-ma.gov/selco/,http://www.shrewsbury-ma.gov/egov/gallery/1341...,1333607030,1333607039,Shrewsbury,"[42.286499, -71.714699]"


In [4]:
df.columns

Index(['a', 'c', 'nk', 'tz', 'gr', 'g', 'h', 'l', 'al', 'hh', 'r', 'u', 't',
       'hc', 'cy', 'll'],
      dtype='object')

In [5]:
df.drop(['c','nk','gr','g','h','l','al','hh'],axis=1,inplace=True)

In [6]:
df.rename(columns={'a':'Browser','tz':'TimeZone','r':'URL From','u':'URL To','t':'TimeStamp Start','hc':'TimeStamp Exist','cy':'City','ll':'Latitude&Longitude'},inplace=True)

In [7]:
#explor the browser info 
df['Browser'][0]

'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.78 Safari/535.11'

In [8]:
#web browser
import httpagentparser
web_list=[] 
for x in df['Browser']:
    web_list.append (httpagentparser.simple_detect(x)[1])

In [9]:
web_ser= pd.Series(web_list)
df['web_browser']=web_ser 

In [10]:
#operating system
os_list=[]
for x in df['Browser']:
    os_list.append(httpagentparser.simple_detect(x)[0])

In [11]:
os_ser = pd.Series(os_list)
df['operating_system']=os_ser

In [12]:
#URL From in short formate
url_list=[]
for x in df['URL From']:
    url=x.split('/')
    if 'direct' in url :
        url_list.append('direct')
    else:
        url_list.append(url[2])

In [13]:
url_ser = pd.Series(url_list)

In [14]:
df['from_url']=url_ser
df['from_url']

0          www.facebook.com
1          www.AwareMap.com
2                      t.co
3                    direct
4     www.shrewsbury-ma.gov
5     www.shrewsbury-ma.gov
6       plus.url.google.com
7          www.facebook.com
8          www.facebook.com
9                      t.co
10                   direct
11                     t.co
Name: from_url, dtype: object

In [15]:
#anothere way to get the shortened URL To in just a line of code
df['to_url']=df['URL To'].str.split(pat=r'/',expand=True)[2]
df['to_url']

0      www.ncbi.nlm.nih.gov
1      www.monroecounty.gov
2          boxer.senate.gov
3             apod.nasa.gov
4     www.shrewsbury-ma.gov
5     www.shrewsbury-ma.gov
6              www.nasa.gov
7              www.nasa.gov
8              www.nasa.gov
9             apod.nasa.gov
10           www.nysdot.gov
11      oversight.house.gov
Name: to_url, dtype: object

In [16]:
#latitude & longitude column data type
for x in df['Latitude&Longitude']:
    print(type(x))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'list'>
<class 'list'>


In [17]:
#apply pd.Series on lists of lat&long
lat_and_long=df['Latitude&Longitude'].apply(pd.Series)
lat_and_long

,0,1
0,42.576698,-70.954903
1,40.218102,-111.613297
2,38.900700,-77.043098
3,-23.549999,-46.616699
4,42.286499,-71.714699
5,42.286499,-71.714699
6,51.116699,15.283300
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [18]:
df['longitude']=lat_and_long[0]
df['latitude']=lat_and_long[1]

    

In [19]:
#TimeStamp when the session started and ended 
df['time_in']=pd.to_datetime(df['TimeStamp Start'])
df['time_out']=pd.to_datetime(df['TimeStamp Exist'])

In [20]:
df.columns

Index(['Browser', 'TimeZone', 'URL From', 'URL To', 'TimeStamp Start',
       'TimeStamp Exist', 'City', 'Latitude&Longitude', 'web_browser',
       'operating_system', 'from_url', 'to_url', 'longitude', 'latitude',
       'time_in', 'time_out'],
      dtype='object')

In [21]:
df.drop(['Browser', 'URL From', 'URL To', 'TimeStamp Start',
       'TimeStamp Exist','Latitude&Longitude'],axis=1,inplace=True)

In [22]:
df.fillna('No Data',inplace=True)
df

,TimeZone,City,web_browser,operating_system,from_url,to_url,longitude,latitude,time_in,time_out
0,America/New_York,Danvers,Chrome 17.0.963.78,Windows 7,www.facebook.com,www.ncbi.nlm.nih.gov,42.576698,-70.954903,1970-01-01 00:00:01.333307030,1970-01-01 00:00:01.333307037
1,America/Denver,Provo,Unknown Browser,Unknown OS,www.AwareMap.com,www.monroecounty.gov,40.218102,-111.613297,1970-01-01 00:00:01.331923249,1970-01-01 00:00:01.308262393
2,America/New_York,Washington,Microsoft Internet Explorer 8.0,Windows 7,t.co,boxer.senate.gov,38.9007,-77.043098,1970-01-01 00:00:01.333407030,1970-01-01 00:00:01.333407035
3,America/Sao_Paulo,Braz,Safari 5.1.2,MacOS Macintosh X 10.6.8,direct,apod.nasa.gov,-23.549999,-46.616699,1970-01-01 00:00:01.333507030,1970-01-01 00:00:01.333507044
4,America/New_York,Shrewsbury,Chrome 17.0.963.79,Windows 7,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,42.286499,-71.714699,1970-01-01 00:00:01.333607030,1970-01-01 00:00:01.333607039
5,America/New_York,Shrewsbury,Chrome 17.0.963.79,Windows 7,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,42.286499,-71.714699,1970-01-01 00:00:01.333707030,1970-01-01 00:00:01.333707048
6,Europe/Warsaw,Luban,Chrome 17.0.963.79,Windows XP,plus.url.google.com,www.nasa.gov,51.116699,15.2833,1970-01-01 00:00:01.333807030,1970-01-01 00:00:01.333807040
7,,No Data,Firefox 4.0.1,Windows 7,www.facebook.com,www.nasa.gov,No Data,No Data,1970-01-01 00:00:01.334007036,1970-01-01 00:00:13.340070364
8,,No Data,Opera 12.00,Ubuntu Linux,www.facebook.com,www.nasa.gov,No Data,No Data,1970-01-01 00:00:01.333907030,1970-01-01 00:00:01.333907042
9,,No Data,Chrome 17.0.963.79,Windows 7,t.co,apod.nasa.gov,No Data,No Data,1970-01-01 00:00:01.334007030,1970-01-01 00:00:01.334007040


In [23]:
df.replace({'':'No data'}, inplace = True)
df

,TimeZone,City,web_browser,operating_system,from_url,to_url,longitude,latitude,time_in,time_out
0,America/New_York,Danvers,Chrome 17.0.963.78,Windows 7,www.facebook.com,www.ncbi.nlm.nih.gov,42.576698,-70.954903,1970-01-01 00:00:01.333307030,1970-01-01 00:00:01.333307037
1,America/Denver,Provo,Unknown Browser,Unknown OS,www.AwareMap.com,www.monroecounty.gov,40.218102,-111.613297,1970-01-01 00:00:01.331923249,1970-01-01 00:00:01.308262393
2,America/New_York,Washington,Microsoft Internet Explorer 8.0,Windows 7,t.co,boxer.senate.gov,38.9007,-77.043098,1970-01-01 00:00:01.333407030,1970-01-01 00:00:01.333407035
3,America/Sao_Paulo,Braz,Safari 5.1.2,MacOS Macintosh X 10.6.8,direct,apod.nasa.gov,-23.549999,-46.616699,1970-01-01 00:00:01.333507030,1970-01-01 00:00:01.333507044
4,America/New_York,Shrewsbury,Chrome 17.0.963.79,Windows 7,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,42.286499,-71.714699,1970-01-01 00:00:01.333607030,1970-01-01 00:00:01.333607039
5,America/New_York,Shrewsbury,Chrome 17.0.963.79,Windows 7,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,42.286499,-71.714699,1970-01-01 00:00:01.333707030,1970-01-01 00:00:01.333707048
6,Europe/Warsaw,Luban,Chrome 17.0.963.79,Windows XP,plus.url.google.com,www.nasa.gov,51.116699,15.2833,1970-01-01 00:00:01.333807030,1970-01-01 00:00:01.333807040
7,No data,No Data,Firefox 4.0.1,Windows 7,www.facebook.com,www.nasa.gov,No Data,No Data,1970-01-01 00:00:01.334007036,1970-01-01 00:00:13.340070364
8,No data,No Data,Opera 12.00,Ubuntu Linux,www.facebook.com,www.nasa.gov,No Data,No Data,1970-01-01 00:00:01.333907030,1970-01-01 00:00:01.333907042
9,No data,No Data,Chrome 17.0.963.79,Windows 7,t.co,apod.nasa.gov,No Data,No Data,1970-01-01 00:00:01.334007030,1970-01-01 00:00:01.334007040


In [25]:
final_df=df.to_csv('G:/Data Science/ITI Data Anaysis/Python/final_df.csv',index=False)

All CSV files must have the following columns
- web_browser
        The web browser that has requested the service
- operating_sys
        operating system that intiated this request
- from_url

        The main URL the user came from

    **note**:

    If the retrived URL was in a long format `http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/wfLQtf`

     make it appear in the file in a short format like this `www.facebook.com`
     
    
- to_url

       The same applied like `to_url`
   
- city

        The city from which the the request was sent
    
- longitude

        The longitude where the request was sent
- latitude

        The latitude where the request was sent

- time_zone
        
        The time zone that the city follow
        
- time_in

        Time when the request started
- time_out
        
        Time when the request is ended
        
        
**NOTE** :

Because that some instances of the file are incomplete, you may encouter some NaN values in your transforamtion. Make sure that the final dataframes have no NaNs at all.